In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tempfile import TemporaryDirectory
import time

import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
from torchvision.models import resnet101, resnet50, resnet18
import torchvision.transforms as transforms
import torchvision.transforms as tt
from torchvision.datasets import CIFAR100 
from torchvision import datasets, models, transforms
from torchinfo import summary

from tqdm import tqdm

In [ ]:
torch.manual_seed(0)

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
!rm -rf /kaggle/working/models/*

In [ ]:
os.makedirs("/kaggle/working/models",exist_ok=True)
os.makedirs("/kaggle/working/remodels",exist_ok=True)

# Compute stats

In [ ]:
stat_train_data = CIFAR100(download=True, root="./data", transform=tt.ToTensor())
stat_test_data = CIFAR100(root="./data", train=False, transform=tt.ToTensor())

In [ ]:
imgs_train = torch.stack([img_t for img_t ,_ in stat_train_data],dim=3)
imgs_test = torch.stack([img_t for img_t ,_ in stat_test_data],dim=3)
imgs = torch.cat((imgs_train, imgs_test), 3)
mean = imgs.view(3,-1).mean(dim=1).numpy()
std = imgs.view(3, -1).std(dim=1).numpy()
print(mean, std)

# Stats and transform

In [ ]:
stats=((0.50736207, 0.4866896, 0.44108862), (0.26748815, 0.2565931, 0.2763085))
train_transform = tt.Compose([
    tt.RandomHorizontalFlip(),
    tt.RandomCrop(32, padding=4, padding_mode="reflect"),
    tt.ToTensor(),
    tt.Normalize(*stats)
])

test_transform = tt.Compose([
    tt.ToTensor(),
    tt.Normalize(*stats)
])

# 100 classes dataset

In [ ]:
train_data = CIFAR100(download=True, root="./data", transform=train_transform)
test_data = CIFAR100(root="./data", train=False, transform=test_transform)

In [ ]:
batch_size=128
num_work=4
train_dl = DataLoader(train_data, batch_size, shuffle=True, num_workers=num_work, pin_memory=True)
test_dl = DataLoader(test_data, batch_size, shuffle=True, num_workers=num_work, pin_memory=True)

In [ ]:
name_list=[]
for nr, name in enumerate(train_data.classes):
    name_list.append([nr,name,train_data.class_to_idx[name]])
    
print(name_list)

In [ ]:
train_data.classes

# 20 superclasses dataset

In [ ]:
class CIFAR100Coarse(torch.utils.data.Dataset):
    def __init__(self, cifar100_dataset) ->None:
        self.cifar100_dataset = cifar100_dataset
        
        # Define the mapping from fine labels to coarse labels
        self.fine_to_coarse={
            0: 4, 1: 1, 2: 14, 3: 8, 4: 0, 5: 6, 6: 7, 7: 7, 8: 18, 9: 3,
            10: 3, 11: 14, 12: 9, 13: 18, 14: 7, 15: 11, 16: 3, 17: 9, 18: 7,
            19: 11, 20: 6, 21: 11, 22: 5, 23: 10, 24: 7, 25: 6, 26: 13, 27: 15,
            28: 3, 29: 15, 30: 0, 31: 11, 32: 1, 33: 10, 34: 12, 35: 14, 36: 16,
            37: 9, 38: 11, 39: 5, 40: 5, 41: 19, 42: 8, 43: 8, 44: 15, 45: 13,
            46: 14, 47: 17, 48: 18, 49: 10, 50: 16, 51: 4, 52: 17, 53: 4, 54: 2,
            55: 0, 56: 17, 57: 4, 58: 18, 59: 17, 60: 10, 61: 3, 62: 2, 63: 12,
            64: 12, 65: 16, 66: 12, 67: 1, 68: 9, 69: 19, 70: 2, 71: 10, 72: 0,
            73: 1, 74: 16, 75: 12, 76: 9, 77: 13, 78: 15, 79: 13, 80: 16, 81: 19,
            82: 2, 83: 4, 84: 6, 85: 19, 86: 5, 87: 5, 88: 8, 89: 19, 90: 18, 91: 1,
            92: 2, 93: 15, 94: 6, 95: 0, 96: 17, 97: 8, 98: 14, 99: 13
        }
        
        # Mapping from superclass labels to superclass names
        self.coarse_label_names = {
            0: 'aquatic mammals',
            1: 'fish',
            2: 'flowers',
            3: 'food containers',
            4: 'fruit and vegetables',
            5: 'household electrical devices',
            6: 'household furniture',
            7: 'insects',
            8: 'large carnivores',
            9: 'large man-made outdoor things',
            10: 'large natural outdoor scenes',
            11: 'large omnivores and herbivores',
            12: 'medium-sized mammals',
            13: 'non-insect invertebrates',
            14: 'people',
            15: 'reptiles',
            16: 'small mammals',
            17: 'trees',
            18: 'vehicles 1',
            19: 'vehicles 2'
        }

    def __getitem__(self, index):
        img, target = self.cifar100_dataset[index]

        # Convert fine label to coarse label
        target = self.fine_to_coarse[target]

        return img, target

    def __len__(self):
        return len(self.cifar100_dataset)

    def get_coarse_label_name(self, label):
        return self.coarse_label_names[label]

    def get_all_coarse_label_names(self):
        return self.coarse_label_names.values()


In [ ]:
train_data_20=CIFAR100Coarse(train_data)
test_data_20=CIFAR100Coarse(test_data)

In [ ]:
batch_size=128
num_work=4
train_dl_20 = DataLoader(train_data_20, batch_size, shuffle=True, num_workers=num_work, pin_memory=True)
test_dl_20 = DataLoader(test_data_20, batch_size, shuffle=True, num_workers=num_work, pin_memory=True)

## Print number of items for each superclass

In [ ]:
train_classes_20={}

for img,label in train_data_20:
    class_name=train_data_20.coarse_label_names[label]
    if class_name not in train_classes_20:
        train_classes_20[class_name]=1
    else:
        train_classes_20[class_name]+=1

In [ ]:
train_classes_20

# Device - GPU cuda

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Model

In [ ]:
model=resnet18(weights=None)

In [ ]:
model=resnet18(weights='IMAGENET1K_V1')

In [ ]:
model.conv1=nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
model.maxpool = nn.Identity()
num_in_ftrs=model.fc.in_features
num_out_ftrs=model.fc.out_features
num_classes=20
model.fc=nn.Linear(num_in_ftrs,num_classes)

In [ ]:
model=model.to(device)

In [ ]:
summary(model,(128,3,32,32))

In [ ]:
model.eval().to(device)

In [ ]:
correct_test, total_test= 0, 0
with torch.no_grad():
    for (inputs, targets) in test_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model_test(inputs)
        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)

accuracy_test = correct_test / total_test
accuracy_test_prc=accuracy_test*100
print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

In [ ]:
model.eval()
correct_test, total_test= 0, 0
all_preds = []
all_labels = []
with torch.no_grad():
    for (inputs, targets) in test_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

accuracy_test = correct_test / total_test
accuracy_test_prc=accuracy_test*100
print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

# Optimizer, scheduler, criterion

In [ ]:
# For optimizer:
max_lr=5e-2
momentum=0.9
weight_decay = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=max_lr, momentum=momentum, weight_decay=weight_decay)

In [ ]:
 scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40,55,60,80,120,140], gamma=0.1)

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=7, min_lr=5e-05)

In [ ]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.3)

# Training and validation loop

In [ ]:
num_epochs=300

In [ ]:
!rm -rf /kaggle/working/models/*

# Train for 20 superclasses

In [ ]:
train_loss=[]
train_acc=[]
test_loss=[]
test_acc=[]
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    data_loader = tqdm(train_dl_20, total=len(train_dl_20), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)


        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        # Progress bar description
        data_loader.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

    average_loss = total_loss / len(train_dl_20)
    accuracy = correct_predictions / total_samples
    
    train_loss.append(average_loss)
    train_acc.append(accuracy)

    # test the model
    correct_test, total_test, total_loss_test = 0, 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl_20:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss_test += loss.item()

            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

        accuracy_test = correct_test / total_test
        average_loss_test = total_loss_test / len(test_dl_20)

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%, Loss on test data: {average_loss_test:.4f} Accuracy on test data: {100 * accuracy_test:.2f} %')
    
    test_loss.append(average_loss_test)
    test_acc.append(accuracy_test)

    model_save_path = '/kaggle/working/models/resnet_cifar10_' + str(epoch+1) + '.pth'
    training_stats = {
      'epoch': epoch+1,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'loss': average_loss,
      'accuracy': accuracy,
      'test_accuracy': accuracy_test,
      'test_loss': average_loss_test,
      'learning_rate': optimizer.param_groups[0]["lr"]
    }
    
#     if epoch%5==0:
#         model_save_path = '/kaggle/working/models/resnet_cifar10_' + str(epoch+1) + '.pth'
#         training_stats = {
#           'epoch': epoch+1,
#           'model_state_dict': model.state_dict(),
#           'optimizer_state_dict': optimizer.state_dict(),
#           'scheduler_state_dict': scheduler.state_dict(),
#           'loss': average_loss,
#           'accuracy': accuracy,
#           'test_accuracy': accuracy_test,
#           'test_loss': average_loss_test,
#           'learning_rate': optimizer.param_groups[0]["lr"]
#         }
    scheduler.step()
    torch.save(training_stats, model_save_path)

# Train for 100 classes

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    data_loader = tqdm(train_dl_20, total=len(train_dl_20), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)


        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        # Progress bar description
        data_loader.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

    average_loss = total_loss / len(train_dl_20)
    accuracy = correct_predictions / total_samples

    # test the model
    correct_test, total_test, total_loss_test = 0, 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl_20:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss_test += loss.item()

            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

        accuracy_test = correct_test / total_test
        average_loss_test = total_loss_test / len(test_dl_20)

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%, Loss on test data: {average_loss_test:.4f} Accuracy on test data: {100 * accuracy_test:.2f} %')
    # run["train/accuracy"].append(accuracy)

    if epoch%5==0:
        model_save_path = '/kaggle/working/models/resnet_cifar10_' + str(epoch+1) + '.pth'
        training_stats = {
          'epoch': epoch+1,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'scheduler_state_dict': scheduler.state_dict(),
          'loss': average_loss,
          'accuracy': accuracy,
          'test_accuracy': accuracy_test,
          'test_loss': average_loss_test,
          'learning_rate': optimizer.param_groups[0]["lr"]
        }
    scheduler.step()
    torch.save(training_stats, model_save_path)

# Save model separately

In [ ]:
model_save_path = '/kaggle/working/resnet_cifar100_' + str(87) + '.pth'
training_stats = {
  'epoch': epoch+1,
  'model_state_dict': model.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  'scheduler_state_dict': scheduler.state_dict(),
  'loss': average_loss,
  'accuracy': accuracy,
  'test_accuracy': accuracy_test,
  'test_loss': average_loss_test,
  'learning_rate': optimizer.param_groups[0]["lr"]
}

In [ ]:
torch.save(training_stats, model_save_path)

# Retraining the previous model for 100 classes - transfer learning from model for 20 superclasses

## Model setup

In [ ]:
saved_model_path='/kaggle/input/pretrained/resnet_cifar100_100classes_e45.pth'

In [ ]:
saved_model_path='/kaggle/input/pretrained/aftertl_ep104.pth'

In [ ]:
saved_model_path="/kaggle/input/cifar-100/cifar100_20classes_ep42.pth"

In [ ]:
saved_model_path='/kaggle/input/cifar-models/cifar100_100classes_40ep.pth' # 73%

In [ ]:
saved_model_path='/kaggle/input/cifar-models/cifar100_100classes_46ep_II.pth'# 83%

In [ ]:
saved_model_path='/kaggle/input/cifar-models/cifar100_100classes_50ep.pth' #83.87

In [ ]:
saved_model_path='/kaggle/input/cifar-models/cifar100_20classes_ep50.pth' #78.09

In [ ]:
saved_model_path='/kaggle/input/cifar-models/cifar100_20classes_ep53.pth'# 83.62

In [ ]:
saved_model_path='/kaggle/input/pretrained-model/resnet_cifar100_transferlearningfor100clasep1415.pth' 

In [ ]:
saved_model_path = '/kaggle/working/resnet_cifar100_87_20cl.pth'

In [ ]:
saved_model_path='/kaggle/input/premodel/resnet_cifar100_87_20cl.pth'

In [ ]:
saved_model_path='/kaggle/input/model25/resnet_cifar100_20classes_e25.pth'

In [ ]:
model_test = resnet18(weights=None)
model_test.conv1=nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
model_test.maxpool = nn.Identity()
num_classes = 100
model_test.fc = nn.Linear(512, num_classes)

In [ ]:
num_in_ftrs=model_test.fc.in_features
print(num_in_ftrs)

## Model weights loading

In [ ]:
checkpoint = torch.load(saved_model_path)
model_test.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
for param in model_test.parameters():
    param.requires_grad = False

In [ ]:
for param in model_test.fc.parameters():
    param.requires_grad = True

In [ ]:
for param in model_test.parameters():
    print(param.requires_grad)

In [ ]:
num_classes=100

In [ ]:
model_test.fc = nn.Linear(512, num_classes)

In [ ]:
model_test.to(device)

In [ ]:
model_test.eval().to(device)

# Confusion matrix for 20 superclasses

In [ ]:
model_test.eval()
correct_test, total_test= 0, 0
all_preds = []
all_labels = []
with torch.no_grad():
    for (inputs, targets) in test_dl_20:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model_test(inputs)
        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

accuracy_test = correct_test / total_test
accuracy_test_prc=accuracy_test*100
print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

In [ ]:
cm = confusion_matrix(all_labels, all_preds)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', linecolor='black',linewidths=0.5)
plt.xlabel('Klasa przewidywana')
plt.ylabel('Klasa prawdziwa')
plt.title('Macierz pomyłek')
plt.savefig('CM_cifar_20klas_nowy_model.png')
plt.show()

## Confusion matrix for 100 classes

In [ ]:
model_test.eval()
correct_test, total_test= 0, 0
all_preds = []
all_labels = []
with torch.no_grad():
    for (inputs, targets) in test_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model_test(inputs)
        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

accuracy_test = correct_test / total_test
accuracy_test_prc=accuracy_test*100
print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

In [ ]:
cm = confusion_matrix(all_labels, all_preds)

In [ ]:
plt.figure(figsize=(35, 30))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', linecolor='black',linewidths=0.5)
plt.xlabel('Klasa przewidywana')
plt.ylabel('Klasa prawdziwa')
plt.title('Macierz pomyłek')
plt.savefig('CM_cifar_100klas.png')
plt.show()

# Optimizer, scheduler etc.

In [ ]:
# For optimizer:
max_lr=7.5e-1
momentum=0.9
weight_decay = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_test.parameters(), lr=max_lr, momentum=momentum, weight_decay=weight_decay)

In [ ]:
print([param.requires_grad for param in model_test.parameters()])

In [ ]:
scheduler=optim.lr_scheduler.MultiStepLR(optimizer, [35,45,55,65,70,75,80,90,100],gamma=0.3)

In [ ]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.3, verbose='True')

In [ ]:
scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3,patience=7, verbose=True)

In [ ]:
!rm -rf /kaggle/working/remodels/*

In [ ]:
train_loss=[]
train_acc=[]
test_loss=[]
test_acc=[]
num_epochs=300
for epoch in range(num_epochs):
    model_test.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    data_loader = tqdm(test_dl, total=len(test_dl), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)


        outputs = model_test(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        # Progress bar description
        data_loader.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

    average_loss = total_loss / len(test_dl)
    accuracy = correct_predictions / total_samples
    
    train_loss.append(average_loss)
    train_acc.append(accuracy)
    
    model_test.eval()
    # test the model
    correct_test, total_test, total_loss_test = 0, 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model_test(inputs)
            loss = criterion(outputs, targets)
            total_loss_test += loss.item()

            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

        accuracy_test = correct_test / total_test
        average_loss_test = total_loss_test / len(test_dl)
    
    test_loss.append(average_loss_test)
    test_acc.append(accuracy_test)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%, Loss on test data: {average_loss_test:.4f} Accuracy on test data: {100 * accuracy_test:.2f} %')

    model_save_path = '/kaggle/working/remodels/aftertl_ep' + str(epoch+1) + '.pth'
    training_stats = {
      'epoch': epoch+1,
      'model_state_dict': model_test.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'loss': average_loss,
      'accuracy': accuracy,
      'test_accuracy': accuracy_test,
      'test_loss': average_loss_test,
      'learning_rate': optimizer.param_groups[0]["lr"]
    }
        
    #scheduler.step()
    scheduler.step(average_loss_test)
    torch.save(training_stats,model_save_path)

# Plots

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Dokładność w procesie uczenia na zbiorze treningowym i testowym")
plt.plot(test_acc[:51],label="Zbiór testowy")
plt.plot(train_acc[:51],label="Zbiór treningowy")
plt.xlabel("Epoka")
plt.ylabel("Dokładność")
plt.xticks(np.arange(0,51,5))
plt.yticks(np.arange(0.0,1.1,0.1))
plt.grid()
plt.legend()
plt.savefig('cifar100_after_accuracy_truegrad.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training and Testing Accuracy")
plt.plot(test_acc,label="Testing accuracy")
plt.plot(train_acc,label="Training accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.xticks(np.arange(0,146,10))
plt.yticks(np.arange(0.4,0.85,0.05))
plt.grid()
plt.legend()
plt.savefig('cifar100_retrainedfor100_acc_50epcheckpoint.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Wartość straty w procesie uczenia na zbiorze treningowym i testowym")
plt.plot(test_loss[:51],label="Zbiór testowy")
plt.plot(train_loss[:51],label="Zbiór treningowy")
plt.xlabel("Epoka")
plt.ylabel("Strata")
plt.xticks(np.arange(0,51,5))
plt.yticks(np.arange(0.0,4.51,0.25))
plt.grid()
plt.legend()
plt.savefig('cifar100_retrainedfor100_loss_50epcheckpoint.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training and Testing Loss")
plt.plot(test_loss,label="Testing loss")
plt.plot(train_loss,label="Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.xticks(np.arange(0,146,10))
plt.yticks(np.arange(0.75,2.256,0.125))
plt.grid()
plt.legend()
plt.savefig('cifar100_retrainedfor100_loss_50epcheckpoint.png')
plt.show()

## Confusion matrix for 100 classes but displayed as 20 superclases

In [ ]:
model_test.eval()
correct_test, total_test= 0, 0
all_preds = []
all_labels = []
with torch.no_grad():
    for (inputs, targets) in test_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model_test(inputs)
        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

accuracy_test = correct_test / total_test
accuracy_test_prc=accuracy_test*100
print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

In [ ]:
fine_to_coarse={
            0: 4, 1: 1, 2: 14, 3: 8, 4: 0, 5: 6, 6: 7, 7: 7, 8: 18, 9: 3,
            10: 3, 11: 14, 12: 9, 13: 18, 14: 7, 15: 11, 16: 3, 17: 9, 18: 7,
            19: 11, 20: 6, 21: 11, 22: 5, 23: 10, 24: 7, 25: 6, 26: 13, 27: 15,
            28: 3, 29: 15, 30: 0, 31: 11, 32: 1, 33: 10, 34: 12, 35: 14, 36: 16,
            37: 9, 38: 11, 39: 5, 40: 5, 41: 19, 42: 8, 43: 8, 44: 15, 45: 13,
            46: 14, 47: 17, 48: 18, 49: 10, 50: 16, 51: 4, 52: 17, 53: 4, 54: 2,
            55: 0, 56: 17, 57: 4, 58: 18, 59: 17, 60: 10, 61: 3, 62: 2, 63: 12,
            64: 12, 65: 16, 66: 12, 67: 1, 68: 9, 69: 19, 70: 2, 71: 10, 72: 0,
            73: 1, 74: 16, 75: 12, 76: 9, 77: 13, 78: 15, 79: 13, 80: 16, 81: 19,
            82: 2, 83: 4, 84: 6, 85: 19, 86: 5, 87: 5, 88: 8, 89: 19, 90: 18, 91: 1,
            92: 2, 93: 15, 94: 6, 95: 0, 96: 17, 97: 8, 98: 14, 99: 13
        }

In [ ]:
all_preds_coarse=[]
for i in range(len(all_preds)):
    all_preds_coarse.append(fine_to_coarse[all_preds[i]])

In [ ]:
all_labels_coarse=[]
for i in range(len(all_preds)):
    all_labels_coarse.append(fine_to_coarse[all_labels[i]])

In [ ]:
cm=confusion_matrix(all_labels_coarse,all_preds_coarse)

In [ ]:
#cm=confusion_matrix(all_labels_coarse,all_preds_coarse)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', linecolor='black',linewidths=0.5)
plt.xlabel('Klasa przewidywana')
plt.ylabel('Klasa prawdziwa')
plt.title('Macierz pomyłek')
#plt.savefig('CM_after_cifar_100.png')
plt.show()

## Confusion matrix - 20 vs 100 classes (just the display, data are from the one for 100 classes) 

In [ ]:
fine_to_new_index = {}
fine_to_new_index_names={}
current_index = 0

# Iterate over each superclass (0 to 19)
for superclass in range(20):
    # Find all fine classes that belong to the current superclass
    fine_classes = [fine for fine, coarse in fine_to_coarse.items() if coarse == superclass]
    # Sort the fine classes for the current superclass
    fine_classes.sort()
    # Map each fine class to a new index
    for fine_class in fine_classes:
        fine_to_new_index[fine_class] = current_index
        fine_to_new_index_names[current_index]=train_data.classes[fine_class]
        current_index += 1

In [ ]:
new_predicted_labels = [fine_to_new_index[pred] for pred in all_preds]

In [ ]:
true_coarse_labels = [fine_to_coarse[label] for label in all_labels]

In [ ]:
conf_matrix = np.zeros((20, 100), dtype=int)

In [ ]:
for true_label, pred_label in zip(true_coarse_labels, new_predicted_labels):
    conf_matrix[true_label, pred_label] += 1

In [ ]:
plt.figure(figsize=(35, 10))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues', linecolor='black',linewidths=0.5, xticklabels=np.arange(100), yticklabels=np.arange(20), vmin=0, vmax=120)
plt.xlabel('Klasa przewidywana')
plt.ylabel('Klasa prawdziwa')
plt.title('Macierz pomyłek - Superklasy oraz wszystkie klasy')
plt.tight_layout()
plt.savefig('macierz_pomyłek_20na100_aftertrl.png')
plt.show()

# Which classes are mostly mistaken in each superclass?

In [ ]:
def find_mistake_details(conf_matrix):
    superclasses = 20
    normal_classes_per_superclass = 5
    result = []

    for superclass in range(superclasses):
        normal_class_indices = list(range(superclass * normal_classes_per_superclass,(superclass + 1) * normal_classes_per_superclass))
        row = conf_matrix[superclass, :]

        sum_superclass_examples = row[normal_class_indices].sum()
        
        masked_row = np.ma.masked_array(row, mask=False)
        masked_row.mask[normal_class_indices] = True

        max_mistake_class_1 = masked_row.argmax()
        max_mistake_count_1 = masked_row[max_mistake_class_1]
        
        masked_row.mask[max_mistake_class_1] = True
        
        max_mistake_class_2 = masked_row.argmax()
        max_mistake_count_2 = masked_row[max_mistake_class_2]
        
        result.append([superclass, sum_superclass_examples, max_mistake_class_1, max_mistake_count_1, max_mistake_class_2, max_mistake_count_2])
    return result


mistake_details = find_mistake_details(conf_matrix)

# DataFrame
columns = ['superclass_index', 'sum_superclass_examples', 'max_mistake_class_1', 'max_mistake_count_1', 'max_mistake_class_2', 'max_mistake_count_2']
df = pd.DataFrame(mistake_details, columns=columns)

print(df)
df.head()

In [ ]:
df.to_csv('mistake_details_cifar_100aftertrl.csv', index=False)

In [ ]:
def find_mistake_details(conf_matrix):
    superclasses = 20
    normal_classes_per_superclass = 5
    result = []

    for superclass in range(superclasses):
        normal_class_indices = list(range(superclass * normal_classes_per_superclass,(superclass + 1) * normal_classes_per_superclass))
        row = conf_matrix[superclass, :]

        sum_superclass_examples = row[normal_class_indices].sum()
        
        masked_row = np.ma.masked_array(row, mask=False)
        masked_row.mask[normal_class_indices] = True
        
        max_mistake_class_1 = masked_row.argmax()
        max_mistake_count_1 = masked_row[max_mistake_class_1]
         
        masked_row.mask[max_mistake_class_1] = True
        
        max_mistake_class_2 = masked_row.argmax()
        max_mistake_count_2 = masked_row[max_mistake_class_2]
        
        result.append([superclass,train_data_20.coarse_label_names[superclass], sum_superclass_examples, max_mistake_class_1,fine_to_new_index_names[max_mistake_class_1], max_mistake_count_1, max_mistake_class_2,fine_to_new_index_names[max_mistake_class_2], max_mistake_count_2])
    
    return result

mistake_details = find_mistake_details(conf_matrix)

columns = ['superclass_index','superclass_name', 'sum_superclass_examples', 'max_mistake_class_1','max_class_1_name', 'max_mistake_count_1', 'max_mistake_class_2','max_class_2_name', 'max_mistake_count_2']
df = pd.DataFrame(mistake_details, columns=columns)


print(df)
df.head()

In [ ]:
plt.figure(figsize=(35, 10))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues', linecolor='black',linewidths=0.5, xticklabels=np.arange(100), yticklabels=np.arange(20),vmin=0, vmax=120)
plt.xlabel('Klasa przewidywana')
plt.ylabel('Klasa prawdziwa')
plt.title('Macierz pomyłek - Superklasy oraz wszystkie klasy')
plt.tight_layout()
plt.savefig('macierz_pomylek_20na100)aftertl.png')
plt.show()

In [ ]:
plt.figure(figsize=(35, 15))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues', linecolor='black',linewidths=0.5, xticklabels=np.arange(100), yticklabels=np.arange(20))
plt.xlabel('Predicted Fine Classes')
plt.ylabel('True Superclasses')
plt.title('Confusion Matrix (Superclasses vs. Fine Classes)')
plt.show()

In [ ]:
model_test.eval()
ep=3
for e in range(ep):
    correct_test, total_test= 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model_test(inputs)
            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

    accuracy_test = correct_test / total_test
    accuracy_test_prc=accuracy_test*100
    print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

In [ ]:
ep=10
for e in range(ep):
    correct_test, total_test= 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model_test(inputs)
            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

    accuracy_test = correct_test / total_test
    accuracy_test_prc=accuracy_test*100
    print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

In [ ]:
model_save_path = '/kaggle/working/resnet_cifar100_transferlearningfor100clasep14' + str(epoch+1) + '.pth'

In [ ]:
training_stats = {
  'epoch': epoch+1,
  'model_state_dict': model_test.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  'scheduler_state_dict': scheduler.state_dict(),
  'loss': average_loss,
  'accuracy': accuracy,
  'test_accuracy': accuracy_test,
  'test_loss': average_loss_test,
  'learning_rate': optimizer.param_groups[0]["lr"]
}

In [ ]:
torch.save(training_stats, model_save_path)

# PCA

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self, base_model):
        super(FeatureExtractor, self).__init__()
        self.features = nn.Sequential(*list(base_model.children())[:-1])

    def forward(self, x):
        return self.features(x)

In [ ]:
saved_model_path='/kaggle/input/cifar-models/cifar100_100classes_50ep.pth'

In [ ]:
checkpoint = torch.load(saved_model_path)
num_classes = 20

net = resnet18(weights=None)
net.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
net.maxpool = nn.Identity()
net.fc = nn.Linear(512, num_classes)
net.load_state_dict(checkpoint['model_state_dict'])
net.to(device)

model_pca = resnet18(weights=None)
model_pca.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model_pca.maxpool = nn.Identity()
model_pca.fc = nn.Linear(512, num_classes)
model_pca.load_state_dict(checkpoint['model_state_dict'])
model_pca.fc=nn.Identity()
model_pca.to(device)

In [ ]:
model_pca = FeatureExtractor(net)
model_pca.to(device)
model_pca.eval()

In [ ]:
counter = 1
features = []
labels = []

with torch.no_grad():
    for (inputs, targets) in train_dl_20:
        inputs, targets = inputs.to(device), targets.to(device)
        for i in range(len(targets)):
            labels.append(targets[i].to('cpu'))
        outputs = model_pca(inputs).to(device)
        for i in range(len(outputs)):
            output = torch.flatten(outputs[i])
            features.append(output.to('cpu'))
        print('Batch ', counter, '/', len(train_dl_20))
        counter = counter + 1

In [ ]:
features_np = [x.numpy() for x in features]
features_np = np.array(features_np)

In [ ]:
print(features_np.shape)
features_np = features_np.reshape(-1, features_np.shape[1])
print(features_np.shape)

In [ ]:
print(features_np.shape)
#features_np = features_np.reshape(-1, features_np.shape[1])
#print(features_np.shape)
labels_np = np.array([y.squeeze().numpy() for y in labels])
print(labels_np)

features_col = ['Feature' + str(i) for i in range(features_np.shape[1])]
df_cifar = pd.DataFrame(features_np, columns = features_col)
df_cifar['Label'] = labels_np

In [ ]:
X = df_cifar.iloc[:, :-1]  #features bo bez ostatniej col
y = df_cifar['Label']  #labels

In [ ]:
pca_model = PCA(n_components=2)
converted_data = pca_model.fit_transform(X)

In [ ]:
df_converted_data = pd.DataFrame(data=converted_data, columns=['Principal Component 1', 'Principal Component 2'])
df_converted_data['Label'] = y

In [ ]:
plt.figure(figsize=(10, 6))
targets = df_converted_data['Label'].unique()
for target in targets:
    plt.scatter(df_converted_data.loc[df_converted_data['Label'] == target, 'Principal Component 1'],
                df_converted_data.loc[df_converted_data['Label'] == target, 'Principal Component 2'],
                alpha=0.5,
                label=target)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of CIFAR Data')
plt.legend()
plt.show()

In [ ]:
# PCA
pca_model = PCA(n_components=3)
converted_data = pca_model.fit_transform(X)

# DataFrame for transformed data
df_converted_data = pd.DataFrame(data=converted_data, columns=['Principal Component 1', 'Principal Component 2', 'Principal Component 3'])
df_converted_data['Label'] = y

# Plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
targets = df_converted_data['Label'].unique()
for target in targets:
    ax.scatter(df_converted_data.loc[df_converted_data['Label'] == target, 'Principal Component 1'],
               df_converted_data.loc[df_converted_data['Label'] == target, 'Principal Component 2'],
               df_converted_data.loc[df_converted_data['Label'] == target, 'Principal Component 3'],
               alpha=0.5,
               label=target)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('PCA of CIFAR Data')
ax.legend()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

In [ ]:
%matplotlib widget 

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot each class separately
targets = df_converted_data['Label'].unique()
for target in targets:
    data = df_converted_data[df_converted_data['Label'] == target]
    ax.scatter(data['Principal Component 1'], data['Principal Component 2'], data['Principal Component 3'], label=target)

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('PCA of CIFAR Data')
ax.legend()

# Use Pandas' interactive features
plt.show()

In [ ]:
# PCA
pca_model = PCA(n_components=3)
converted_data = pca_model.fit_transform(X)

# DataFrame for transformed data
df_converted_data = pd.DataFrame(data=converted_data, columns=['Principal Component 1', 'Principal Component 2', 'Principal Component 3'])
df_converted_data['Label'] = y

# Plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot each class separately
targets = df_converted_data['Label'].unique()
for target in targets:
    data = df_converted_data[df_converted_data['Label'] == target]
    ax.scatter(data['Principal Component 1'], data['Principal Component 2'], data['Principal Component 3'], label=target)

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('PCA of CIFAR Data')
ax.legend()

# Enable interactive mode
plt.ion()
plt.show()

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Plot using Plotly
fig = go.Figure()

# Plot each class separately
targets = df_converted_data['Label'].unique()
for target in targets:
    data = df_converted_data[df_converted_data['Label'] == target]
    fig.add_trace(go.Scatter3d(
        x=data['Principal Component 1'],
        y=data['Principal Component 2'],
        z=data['Principal Component 3'],
        mode='markers',
        marker=dict(size=5),
        name='Class'
    ))

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title='Principal Component 1',
        yaxis_title='Principal Component 2',
        zaxis_title='Principal Component 3',
        title='PCA of CIFAR Data'
    )
)

fig.show()

In [ ]:
fig = go.Figure()

# Plot each class separately
targets = df_converted_data['Label'].unique()
for target in targets:
    data = df_converted_data[df_converted_data['Label'] == target]
    fig.add_trace(go.Scatter3d(
        x=data['Principal Component 1'],
        y=data['Principal Component 2'],
        z=data['Principal Component 3'],
        mode='markers',
        marker=dict(size=5),
        name=target
    ))

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title='Principal Component 1',
        yaxis_title='Principal Component 2',
        zaxis_title='Principal Component 3',
        title='PCA of CIFAR Data'
    )
)

fig.show()

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
class_variance = df_converted_data.groupby('Label').apply(lambda x: x.iloc[:, :-1].var())
total_variance = df_converted_data.iloc[:, :-1].var()

# Compute the separability ratio for each class
separability_ratio = class_variance.div(total_variance)
print("\nSeparability ratio:\n", separability_ratio)

In [ ]:
# Obliczenie centroidów
centroids = df_converted_data.groupby('Label').mean()

# Odległość między centroidami
centroid_distances = euclidean_distances(centroids)

# Współczynnik separowalności
within_class_variances = df_converted_data.groupby('Label').apply(lambda x: np.var(x, ddof=1))
total_within_class_variance = within_class_variances.sum()
between_class_variance = np.var(centroids, ddof=1)
separability_ratio = between_class_variance / total_within_class_variance

print("Odległość między centroidami:")
print(centroid_distances)
print("\nWspółczynnik separowalności:")
print(separability_ratio)

In [ ]:
convereted_data = pca_model.fit_transform(df_cifar.iloc[:,:-1])
df_converted_data = pd.DataFrame(data = convereted_data, columns = ['Principal Component 1', 'Principal Component 2'])
df_converted_data['Label'] = labels_np

print(df_converted_data.head())

In [ ]:
classes=list(range(1,21))